In [714]:
import pandas as pd
import hvplot.pandas
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import os
import plotly.express as px


In [715]:
# Creating obect for the excel files
# apparel_xl = pd.read_excel('Apparel.xlsx')
# take_out_xl = pd.read_excel('Food_away_from_home.xlsx')
# housing_xl = pd.read_excel('Philadelphia_housing.xlsx')
# recreation_xl = pd.read_excel('Philly_Recreation.xlsx')

In [716]:
# Creating csv's out of the excel object
# apparel_csv = apparel_xl.to_csv('Philly_Apparel.csv')
# take_out_csv = take_out_xl.to_csv('Philly_take_out.csv')
# housing_csv = housing_xl.to_csv('Philly_housing.csv')
# recreation_csv = recreation_xl.to_csv('Philly_Recreation.csv')

In [717]:
# Creating the dataframes from the csv's
Philly_apparel = pd.read_csv('apparel_clean.csv',index_col='Year')
Philly_commercial_food = pd.read_csv('food_clean.csv',index_col='Year')
Philly_housing = pd.read_csv('housing_clean.csv',index_col='Year')


In [718]:
# Cleaning the df columns to only include "Annual"
Philly_apparel = Philly_apparel[['Annual']]
Philly_commercial_food = Philly_commercial_food[['Annual']]
Philly_housing = Philly_housing[['Annual']]



In [719]:
# Rename columns
Philly_apparel.columns = ['PHL Apparel Annual']
Philly_commercial_food.columns = ['PHL Commercial Food Annual']
Philly_housing.columns = ['PHL Housing Annual']
# Philly_recreation.columns = ['PHL Recreation Annual']


In [720]:
# Combine the dataframes
df_list = [Philly_apparel,Philly_commercial_food,Philly_housing]

combined_df = pd.concat(df_list, axis=1, join='inner')


In [721]:
# Df for Percent change year to year
pct_change_df = combined_df.pct_change()



In [722]:
# load env file
load_dotenv()

True

In [723]:
# Retreive API keys
public_key = os.getenv("ALPACA_API_KEY")
private_key = os.getenv("ALPACA_SECRET_KEY")
type(private_key)

str

In [724]:
# Create alpacas connection object
alpacas = tradeapi.REST(public_key, private_key, api_version="v2")

In [725]:
start = pd.Timestamp("2012-01-01", tz="America/New_York").isoformat()
end = pd.Timestamp("2021-12-31", tz="America/New_York").isoformat()
timeframe="1Day"

In [726]:
real_estate = ["TSCO","DHI","LOW","LGIH"]
apparel = ["NKE", "UA","GPS", "LULU", "JWN"]
pleasure_food = ["MCD","YUM", "DRI", "QSR"]

def alpacas_api(symbols,timeframe, start,end):
    dataframe = alpacas.get_bars(symbols, timeframe=timeframe, start=start, end=end).df
    new_df = dataframe[['open','close','trade_count', 'symbol']]

    return new_df

Real_Estate_stocks = alpacas_api(real_estate, timeframe,start,end)
Apparel_stocks = alpacas_api(apparel, timeframe,start,end)
Food_stocks = alpacas_api(pleasure_food, timeframe,start,end)


In [727]:
# Create pivot tables for closing prices
Real_Estate_close = Real_Estate_stocks.pivot_table(values="close", index="timestamp",columns="symbol")
Apparel_close = Apparel_stocks.pivot_table(values="close", index="timestamp",columns="symbol")
Food_close = Food_stocks.pivot_table(values="close", index="timestamp",columns="symbol")

In [728]:
# Get the percetn changes for each column
apparel_pct_change = combined_df["PHL Apparel Annual"].pct_change()
Food_pct_change = combined_df["PHL Commercial Food Annual"].pct_change()
Housing_pct_change = combined_df["PHL Housing Annual"].pct_change()

In [729]:
# compare the cpi among cities, do the cities face a similar rate of inflation?
# What industries become more expensive quicker in which cities 

In [730]:
def concat_stocks_cpi(stock, cpi):
    df = pd.concat([stock,cpi], axis=1, join="inner")
    return df 

In [731]:
# Real_Estate_stocks.index = Real_Estate_stocks.index.year
Real_Estate_stocks = Real_Estate_stocks.resample("Y").mean()

Real_estate_stock_annual = Real_Estate_stocks["close"].pct_change()
Real_estate_stock_annual.index = Real_estate_stock_annual.index.year
Real_estate_stock_annual
# Housing_pct_change

timestamp
2015         NaN
2016   -0.022360
2017    0.029060
2018    0.231439
2019    0.170760
2020    0.275063
2021    0.536315
Name: close, dtype: float64

In [732]:
def reformat_data(dataframe):
    dataframe = dataframe.resample("Y").mean()

    dataframe = dataframe["close"].pct_change()
    dataframe.index =dataframe.index.year
    return dataframe

Apparel_stock_annual = reformat_data(Apparel_stocks)
Food_stock_annual = reformat_data(Food_stocks)

In [733]:
# IMPORTANT: CPI must be second dataframe entered into concat

def concat_stocks_cpi(stock, cpi):
    df = pd.concat([stock,cpi], axis=1, join="inner")
    return df 

Housing_combined = concat_stocks_cpi(Real_estate_stock_annual, Housing_pct_change).dropna()
Housing_combined.columns = ['Real Estate stocks', "Philly housing CPI"]
Housing_plot = Housing_combined.hvplot.line(xlabel='Year', ylabel="Annual Percent Change")



In [734]:
apparel_combined = concat_stocks_cpi(Apparel_stock_annual,apparel_pct_change).dropna()
apparel_combined.columns = ['Apparel stocks', "Philly Apparel CPI"]
apparel_plot = apparel_combined.hvplot.line(xlabel='Year', ylabel="Annual Percent Change")
apparel_plot

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [735]:
food_combined = concat_stocks_cpi(Food_stock_annual,Food_pct_change).dropna()
food_combined.columns = ['Commerical Food stocks', "Philly Commerical Food CPI"]
food_plot = food_combined.hvplot.line(xlabel='Year', ylabel="Annual Percent Change")


In [736]:
combined_charts = food_plot + Housing_plot + apparel_plot
combined_charts
# create a dropdown menu

:Layout
   .NdOverlay.I   :NdOverlay   [Variable]
      :Curve   [index]   (value)
   .NdOverlay.II  :NdOverlay   [Variable]
      :Curve   [index]   (value)
   .NdOverlay.III :NdOverlay   [Variable]
      :Curve   [index]   (value)

In [743]:
def extract_max(df):
    return df[df.iloc[:, 1] == df.iloc[:, 1].max()]

def extract_min(df):
     return df[df.iloc[:, 1] == df.iloc[:, 1].min()]

def graph_max(df):
    bar = extract_max(df)
    graph = bar.hvplot.bar(ylabel="Percentage change", xlabel="Year", title="CPI Best year and cooresponding stock data", rot=90)
    return graph

def graph_min(df):
    bar = extract_min(df)
    graph = bar.hvplot.bar(ylabel="Percentage change", xlabel="Year", title="CPI Best year and cooresponding stock data", rot=90)
    return graph




:Bars   [index,Variable]   (value)

In [738]:
# Question 3, relationship between trade count and high periods of inflation

def reformat_data_trade_count(dataframe):
    dataframe = dataframe.resample("Y").mean()

    dataframe = dataframe["trade_count"]
    dataframe.index =dataframe.index.year
    return dataframe

def combine_cpi_trade_count(df_stocks, df_cpi):
    reformed = reformat_data_trade_count(df_stocks)
    trade = concat_stocks_cpi(reformed, df_cpi)
    return trade

def graph_max_trade_count(df_stocks, df_cpi,column_name):
    data = combine_cpi_trade_count(df_stocks,df_cpi).sort_values(by=f"{column_name}")
    highest_inflation = data.iloc[-3:]
    plot = px.scatter(highest_inflation,x=f"{column_name}", y="trade_count")
    return plot






Philly_real_estate_scatter = graph_max_trade_count(Real_Estate_stocks,Housing_pct_change,"PHL Housing Annual")
Philly_apparel_scatter = graph_max_trade_count(Apparel_stocks, apparel_pct_change, "PHL Apparel Annual")
Philly_food_scatter = graph_max_trade_count(Food_stocks, Food_pct_change, "PHL Commercial Food Annual")
